# IBM Applied Data Science Capstone Project
### Week 4 Final Report
**_Opening a new Chinese Restaurant in Mumbai, India_**
- Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Chinese Restaurant
***
### 1. Import libraries

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup # library to parse HTML and XML documents

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - anaconda/linux-64::grpcio==1.16.1=py36hf8bcb03_1 -> openssl[version='>=1.1.1,<1.1.2.0a0']

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


WARNING conda.base.context:use_only_tar_bz2(632): Co

### 2. Scrap data from Wikipedia page into a DataFrame

In [10]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Mumbai").text

In [11]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [12]:
# create a list to store neighborhood data
neighborhoodList = []

In [13]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [15]:
# create a new DataFrame from the list
mum_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mum_df = mum_df.drop([0], axis=0)

mum_df.head()

,Neighborhood
1,Aarey Milk Colony
2,Agripada
3,Altamount Road
4,"Amboli, Mumbai"
5,Amrut Nagar


In [17]:
# print the number of rows of the dataframe
mum_df.shape

(136, 1)

### 3. Get the geographical coordinates

In [24]:
!pip install geocoder
import geocoder # to get coordinates

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

     |████████████████████████████████| 102kB 17.3MB/s ta 0:00:01


In [25]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in mum_df["Neighborhood"].tolist() ]

In [26]:
coords

[[19.166115394396662, 72.85940270774424],
 [18.976260000000025, 72.82615000000004],
 [18.96433788604836, 72.80783688604835],
 [18.940170000000023, 72.83486000000005],
 [19.14510000000007, 72.84675000000004],
 [19.026350000000036, 72.86634000000004],
 [19.042830000000038, 72.92734000000007],
 [19.095240000000047, 72.89255000000003],
 [18.920250335276123, 72.82611517568816],
 [19.06293000000005, 72.92666000000008],
 [18.935410000000047, 72.83981000000006],
 [19.054220000000043, 72.84019000000006],
 [19.062070000000062, 72.86849000000007],
 [19.16784000000007, 72.83293000000003],
 [18.95130000000006, 72.82930000000005],
 [19.140256823437614, 72.8047790882812],
 [18.969030000000032, 72.80384000000004],
 [18.980740000000026, 72.84075000000007],
 [19.138910000000067, 72.93818000000005],
 [18.9497029367247, 72.82732610600179],
 [18.996790000000033, 72.84654000000006],
 [19.110160000000064, 72.89174000000008],
 [18.985940000000028, 72.83117000000004],
 [18.947690000000023, 72.82697000000007],


In [27]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [28]:
# merge the coordinates into the original dataframe
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

In [34]:
# Dropping all NaN Values
mum_df = mum_df.dropna()

In [35]:
# check the neighborhoods and the coordinates
print(mum_df.shape)
mum_df

(135, 3)


,Neighborhood,Latitude,Longitude
1,Aarey Milk Colony,18.976260,72.826150
2,Agripada,18.964338,72.807837
3,Altamount Road,18.940170,72.834860
4,"Amboli, Mumbai",19.145100,72.846750
5,Amrut Nagar,19.026350,72.866340
6,Antop Hill,19.042830,72.927340
7,Anushakti Nagar,19.095240,72.892550
8,Asalfa,18.920250,72.826115
9,Badhwar Park,19.062930,72.926660
10,Baiganwadi,18.935410,72.839810


In [36]:
# save the DataFrame as CSV file
mum_df.to_csv("mum_df.csv", index=False)

### 4. Create a map of Mumbai with neighborhoods superimposed on top

In [37]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [38]:
# create map of Toronto using latitude and longitude values
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [39]:
# save the map as HTML file
map_mum.save('map_mum.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [40]:
# define Foursquare Credentials and Version
CLIENT_ID = 'VWSVTBUTOESCTRAGOUTXTVSXYBFJ4IH4SDNSPJNET2PLF2U5' # your Foursquare ID
CLIENT_SECRET = '4V14NPMN1HDAISDS0OUAXT4LXE5BOXIR43UQUDXGML5HKVV1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VWSVTBUTOESCTRAGOUTXTVSXYBFJ4IH4SDNSPJNET2PLF2U5
CLIENT_SECRET:4V14NPMN1HDAISDS0OUAXT4LXE5BOXIR43UQUDXGML5HKVV1


In [41]:
# Now, let's get the top 100 venues that are within a radius of 2000 meters.

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [42]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(10987, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Aarey Milk Colony,18.97626,72.82615,Tote On The Turf,18.980266,72.820294,Nightclub
1,Aarey Milk Colony,18.97626,72.82615,Mahalaxmi Race Course (Royal Western India Tur...,18.980535,72.818588,Club House
2,Aarey Milk Colony,18.97626,72.82615,Celejor,18.975844,72.823679,Bakery
3,Aarey Milk Colony,18.97626,72.82615,Neel,18.980407,72.820403,Indian Restaurant
4,Aarey Milk Colony,18.97626,72.82615,Willingdon Sports Club,18.976925,72.815256,Golf Course


In [43]:
# Let's check how many venues were returned for each neighorhood

venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Aarey Milk Colony,74,74,74,74,74,74
Agripada,94,94,94,94,94,94
Altamount Road,100,100,100,100,100,100
"Amboli, Mumbai",89,89,89,89,89,89
Amrut Nagar,86,86,86,86,86,86
Antop Hill,19,19,19,19,19,19
Anushakti Nagar,100,100,100,100,100,100
Asalfa,100,100,100,100,100,100
Badhwar Park,9,9,9,9,9,9


In [44]:
# Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 233 uniques categories.


In [45]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Nightclub', 'Club House', 'Bakery', 'Indian Restaurant',
       'Golf Course', 'Middle Eastern Restaurant', 'Restaurant',
       'History Museum', 'Scenic Lookout', 'Music Venue', 'Café', 'Hotel',
       'Juice Bar', 'Bengali Restaurant', 'Chinese Restaurant', 'Spa',
       'Bar', 'Deli / Bodega', 'Ice Cream Shop', 'Dessert Shop',
       'Fast Food Restaurant', 'Cupcake Shop', 'Planetarium',
       'Italian Restaurant', 'Clothing Store', 'Shopping Mall', 'Garden',
       'Art Gallery', 'Punjabi Restaurant', 'Snack Place',
       'Mediterranean Restaurant', 'Electronics Store',
       'Furniture / Home Store', 'Pizza Place', 'Theater', 'Coffee Shop',
       'Multiplex', 'Zoo', 'Lounge', 'Antique Shop', 'Farmers Market',
       'Brewery', 'Bookstore', 'Donut Shop', 'Sandwich Place',
       'Salon / Barbershop', 'Park', 'Gym / Fitness Center', 'Stadium',
       'Japanese Restaurant'], dtype=object)

In [48]:
# check if the results contain "Chinese Restaurant"
"Chinese Restaurant" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [50]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(10987, 234)


,Neighborhoods,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Campground,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [51]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(135, 234)


,Neighborhoods,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Campground,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.013514,0.000000,0.027027,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.067568,0.000000,0.040541,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040541,0.00000,0.00,0.000000,0.027027,0.013514,0.013514,0.000000,0.027027,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.013514,0.000000,0.013514,0.000000,0.000000,0.040541,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.013514,0.000000,0.000000,0.000000,0.013514

In [56]:
len(mum_grouped[mum_grouped["Chinese Restaurant"] > 0])

118

In [57]:
# Create a new DataFrame for Shopping Mall data only

mum_cr = mum_grouped[["Neighborhoods","Chinese Restaurant"]]

In [58]:
mum_cr.head()

,Neighborhoods,Chinese Restaurant
0,Aarey Milk Colony,0.027027
1,Agripada,0.021277
2,Altamount Road,0.030000
3,"Amboli, Mumbai",0.067416
4,Amrut Nagar,0.023256


### 7. Cluster Neighborhoods

#### Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [59]:
# set number of clusters
mclusters = 3

mum_clustering = mum_cr.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=mclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 0, 2, 0, 2, 2, 1, 2], dtype=int32)

In [60]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mum_merged = mum_cr.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [61]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Chinese Restaurant,Cluster Labels
0,Aarey Milk Colony,0.027027,2
1,Agripada,0.021277,2
2,Altamount Road,0.030000,2
3,"Amboli, Mumbai",0.067416,0
4,Amrut Nagar,0.023256,2


In [62]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() # check the last columns!

(135, 5)


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.027027,2,18.976260,72.826150
1,Agripada,0.021277,2,18.964338,72.807837
2,Altamount Road,0.030000,2,18.940170,72.834860
3,"Amboli, Mumbai",0.067416,0,19.145100,72.846750
4,Amrut Nagar,0.023256,2,19.026350,72.866340


In [63]:
# sort the results by Cluster Labels
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(135, 5)


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
63,Kamathipura,0.058824,0,19.120160,72.937050
97,Nariman Point,0.062500,0,18.906460,72.814930
101,Orlem,0.050000,0,19.150540,72.830810
55,"Hiranandani Gardens, Mumbai",0.052632,0,19.249170,72.850230
119,Shivaji Park,0.068182,0,19.028750,72.837550
118,Shimpoli,0.068182,0,19.028750,72.837550
117,"Shastri Nagar, Goregaon",0.052083,0,19.222680,72.843280
108,Powai,0.050000,0,19.014200,72.827760
53,Guru Tegh Bahadur Nagar,0.060000,0,19.019710,72.847430
32,D.N. Nagar,0.050000,0,19.019920,72.840870


#### Finally, let's visualize the resulting clusters

In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(mclusters)
ys = [i+x+(i*x)**2 for i in range(mclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [65]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [66]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
63,Kamathipura,0.058824,0,19.120160,72.937050
97,Nariman Point,0.062500,0,18.906460,72.814930
101,Orlem,0.050000,0,19.150540,72.830810
55,"Hiranandani Gardens, Mumbai",0.052632,0,19.249170,72.850230
119,Shivaji Park,0.068182,0,19.028750,72.837550
118,Shimpoli,0.068182,0,19.028750,72.837550
117,"Shastri Nagar, Goregaon",0.052083,0,19.222680,72.843280
108,Powai,0.050000,0,19.014200,72.827760
53,Guru Tegh Bahadur Nagar,0.060000,0,19.019710,72.847430
32,D.N. Nagar,0.050000,0,19.019920,72.840870


#### Cluster 1

In [67]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
81,Mahim,0.000000,1,19.028280,72.897459
49,Girgaon,0.000000,1,19.172270,72.869670
46,Four Bungalows,0.000000,1,19.140932,72.882604
52,Gowalia Tank,0.011494,1,19.038360,72.864000
103,Pali Hill,0.010000,1,19.063260,72.829490
77,Lokhandwala Complex,0.000000,1,19.146800,72.790640
102,Oshiwara,0.010000,1,19.067720,72.825220
69,Koliwada,0.000000,1,18.332630,72.968690
94,Nagpada,0.000000,1,19.419990,72.814030
65,Kemps Corner,0.010000,1,19.084630,72.826810


#### Cluster 2

In [70]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
87,"Marine Drive, Mumbai",0.020000,2,18.943430,72.823190
86,Manori,0.020000,2,18.946660,72.824560
129,Virar,0.021277,2,19.017160,72.858130
88,Marine Lines,0.030000,2,19.119140,72.882820
126,Uran,0.020000,2,18.967136,72.827300
125,Umarkhadi,0.033333,2,19.068640,72.898460
131,Walkeshwar,0.040000,2,18.943256,72.826694
91,Mazagaon,0.030000,2,18.940170,72.834860
128,Vidyavihar,0.021505,2,19.016570,72.858530
105,Parel,0.021505,2,18.964716,72.807557


#### Observations:
Most of the Chinese restaurants are concentrated in the central area of Mumbai city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number of Chinese restaurants in the neighborhood. This represents a great opportunity and high potential areas to open new Chinese restaurants as there is very little to no competition from existing restaurants. Meanwhile, Chinese restaurants in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of Chinese restaurants. From another perspective, this also shows that the oversupply of Chinese restaurants mostly happened in the central area of the city, with the suburb area still have very few Chinese restaurants. Therefore, this project recommends restaurant owners to capitalize on these findings to open new Chinese Restaurants in neighborhoods in cluster 1 with little to no competition. Restaurant owners with unique selling propositions to stand out from the competition can also open new Chinese restaurants in neighborhoods in cluster 0 with moderate competition. Lastly, restaurant owners are advised to avoid neighborhoods in cluster 2 which already have high concentration of Chinese restaurants and suffering from intense competition.